In [9]:

# import include nesscessary
import cv2
from PIL import Image, ImageOps
import argparse
import cv2
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import numpy
import cv2
import numpy as np
import imutils


In [10]:
img = cv2.imread("sudoku2.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (7, 7), 0)

adap_thresh = cv2.adaptiveThreshold(
    blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 3)
cv2.imwrite("output.jpg", adap_thresh)


True

In [11]:
def find_board(img):
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.imread('output.jpg')
    # Loại bỏ nhiễu
    bfilter = cv2.bilateralFilter(gray, 13, 20, 20)
    # Phát hiện các cạnh của hình ảnh   
    edged = cv2.Canny(bfilter, 30, 180)
    # Tìm các điểm liên tục của các cạnh 
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE,
    cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    newimg = cv2.drawContours(img.copy(), contours, -1, (0, 255, 0), 3)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:15]
    location = None
    # result = None
    # Finds rectangular contour
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 15, True)
        if len(approx) == 4:
            location = approx
            break
    cv2.imshow("Board", img)
    cv2.waitKey(0)
    result = get_perspective(img, location)
    return result, location

In [12]:
def get_perspective(img, location, height = 900, width = 900):

    pts1 = np.float32([location[0], location[3], location[1], location[2]])
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    result = cv2.warpPerspective(img, matrix, (width, height))
    return result


In [13]:
board, location = find_board(img)
# board = cv2.rotate(board, cv2.ROTATE_90_CLOCKWISE)
cv2.imshow("Board", board)
cv2.waitKey(0)
# print(board)

-1

In [14]:
# split the board into 81 individual images
def split_boxes(board):
    rows = np.vsplit(board,9)
    boxes = []
    id = 0
    for r in rows:
        cols = np.hsplit(r,9)
        for box in cols:
            box = cv2.resize(box, (input_size, input_size))/255.0
            # cv2.imshow("Splitted block", box)
            # cv2.imwrite(f"{id}.jpg", box*255)
            id+=1
    
            # c
            boxes.append(box*255)
    cv2.destroyAllWindows()
    return boxes

def displayNumbers(img, numbers, color=(0, 255, 0)):
    W = int(img.shape[1]/9)
    H = int(img.shape[0]/9)
    for i in range (9):
        for j in range (9):
            if numbers[(j*9)+i] !=0:
                cv2.putText(img, str(numbers[(j*9)+i]), (i*W+int(W/2)-int((W/4)), int((j+0.7)*H)), cv2.FONT_HERSHEY_COMPLEX, 2, color, 2, cv2.LINE_AA)
    return img

In [15]:
input_size = 48
gray = cv2.cvtColor(board, cv2.COLOR_BGR2GRAY)
# print(gray.shape)
rois = split_boxes(gray)

In [18]:
ans = []
for i in range(len(rois)):
    gray = rois[i].astype('uint8')
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 5, 3)
    # thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    sum = np.sum(thresh[14:34, 14:34])
    if sum > 500:
        ans.append('x')
    else :
        ans.append('_')
    print(f'{i}.jpg = {sum}')

0.jpg = 28815
1.jpg = 21675
2.jpg = 0
3.jpg = 0
4.jpg = 37995
5.jpg = 0
6.jpg = 42585
7.jpg = 0
8.jpg = 0
9.jpg = 0
10.jpg = 0
11.jpg = 0
12.jpg = 0
13.jpg = 0
14.jpg = 40035
15.jpg = 24990
16.jpg = 0
17.jpg = 0
18.jpg = 40545
19.jpg = 0
20.jpg = 44115
21.jpg = 35700
22.jpg = 23460
23.jpg = 0
24.jpg = 0
25.jpg = 36465
26.jpg = 39270
27.jpg = 0
28.jpg = 0
29.jpg = 21420
30.jpg = 0
31.jpg = 0
32.jpg = 35955
33.jpg = 0
34.jpg = 41055
35.jpg = 0
36.jpg = 42075
37.jpg = 0
38.jpg = 42840
39.jpg = 0
40.jpg = 33660
41.jpg = 0
42.jpg = 48195
43.jpg = 0
44.jpg = 33915
45.jpg = 0
46.jpg = 38760
47.jpg = 0
48.jpg = 33660
49.jpg = 0
50.jpg = 0
51.jpg = 45900
52.jpg = 0
53.jpg = 0
54.jpg = 33915
55.jpg = 48450
56.jpg = 0
57.jpg = 0
58.jpg = 42585
59.jpg = 0
60.jpg = 39270
61.jpg = 0
62.jpg = 41310
63.jpg = 0
64.jpg = 0
65.jpg = 47175
66.jpg = 22440
67.jpg = 0
68.jpg = 0
69.jpg = 0
70.jpg = 0
71.jpg = 31620
72.jpg = 0
73.jpg = 0
74.jpg = 20655
75.jpg = 0
76.jpg = 48195
77.jpg = 0
78.jpg = 34425
79.jp

In [19]:
board_num = np.array(ans).reshape(9, 9)
board_num

array([['x', 'x', '_', '_', 'x', '_', 'x', '_', '_'],
       ['_', '_', '_', '_', '_', 'x', 'x', '_', '_'],
       ['x', '_', 'x', 'x', 'x', '_', '_', 'x', 'x'],
       ['_', '_', 'x', '_', '_', 'x', '_', 'x', '_'],
       ['x', '_', 'x', '_', 'x', '_', 'x', '_', 'x'],
       ['_', 'x', '_', 'x', '_', '_', 'x', '_', '_'],
       ['x', 'x', '_', '_', 'x', '_', 'x', '_', 'x'],
       ['_', '_', 'x', 'x', '_', '_', '_', '_', 'x'],
       ['_', '_', 'x', '_', 'x', '_', 'x', '_', 'x']], dtype='<U1')

In [112]:

mat = np.matrix(board_num)
with open('output.txt','wb') as f:
    for line in mat:
        np.savetxt(f, line, fmt='%s')